In [55]:
import pandas as pd

In [56]:
student_df = pd.read_csv("../data_raw/student_info.csv")
tuition_df = pd.read_csv("../data_raw/tuition_fees.csv")
payment_df = pd.read_csv("../data_raw/payment_records.csv")


In [57]:
students_by_program = (
    student_df
    .groupby("program_type")
    .size()
    .reset_index(name="total_students")
)


In [59]:
print(students_by_program)


       program_type  total_students
0            C L C                9
1               CLC               8
2         Chinh quy               7
3         Chính quy              11
4        Lien thong               8
5        Liên thông               5
6   Vua lam vua hoc               5
7   Vừa làm vừa học              10
8        chinh  quy              11
9  vua lam  vua hoc               6


In [61]:
total_tuition_by_semester = (
    tuition_df
    .groupby("semester")["tuition_amount"]
    .sum()
    .reset_index(name="total_tuition")
)
print(total_tuition_by_semester)



    semester                                      total_tuition
0     2024-1  8.000.00010000000100000008000000100000008.000....
1     2024-2   9,000,000 80000009000000900000080000008000000...
2     2025-1  900000010000000800000010.000.000đ1000000010.00...
3  HK1- 2025  8.000.00010.000.000đ90000008.000.00010000000 9...
4   HK1-2024  8.000.00010.000.000đ8.000.000 9,000,000 100000...
5   HK2-2024  8.000.000100000008.000.000 9,000,000 100000009...


In [64]:
# Tự tìm cột tiền thanh toán
pay_col = [c for c in payment_df.columns if "amount" in c.lower()][0]

total_paid_by_semester = (
    payment_df
    .groupby("semester")[pay_col]
    .sum()
    .reset_index(name="total_paid")
)

print(total_paid_by_semester)


    semester                                         total_paid
0     2024-1  10.000.000đ8000000-50000010000000-50000010.000...
1     2024-2  -500000-50000080000004000000 8,000,000 4.000.0...
2     2025-1  40000009000000 8,000,000 80000009000000 8,000,...
3  HK1- 2025  1000000090000001000000010000000 8,000,000 9000...
4   HK1-2024  9000000 8,000,000 1000000040000004.000.0001000...
5   HK2-2024  4000000 8,000,000 4000000-500000100000004.000....


In [65]:
merged = pd.merge(
    tuition_df,
    payment_df,
    on=["student_id", "semester"],
    how="left",
    suffixes=("", "_pay")
)


In [66]:
pay_col_merged = [c for c in merged.columns if "amount" in c.lower() and c != "tuition_amount"][0]


In [67]:
merged[pay_col_merged] = merged[pay_col_merged].fillna(0)


In [68]:
summary = (
    merged
    .groupby(["student_id", "semester"], as_index=False)
    .agg(
        tuition_amount=("tuition_amount", "sum"),
        paid_amount=(pay_col_merged, "sum")
    )
)

print(summary.head())


  student_id   semester tuition_amount paid_amount
0     SV3004     2024-1        9000000           0
1     SV3010  HK1- 2025     9,000,000      8000000
2     SV3013     2024-1     9,000,000            0
3     SV3014  HK1- 2025        9000000           0
4     SV3014   HK1-2024      8.000.000           0


In [73]:
summary["paid_amount"] = summary["paid_amount"].fillna(0)

unpaid_or_partial = summary[
    summary["paid_amount"] != summary["tuition_amount"]
]

print(unpaid_or_partial)


    student_id   semester tuition_amount paid_amount
0       SV3004     2024-1        9000000           0
1       SV3010  HK1- 2025     9,000,000      8000000
2       SV3013     2024-1     9,000,000            0
3       SV3014  HK1- 2025        9000000           0
4       SV3014   HK1-2024      8.000.000           0
..         ...        ...            ...         ...
102     sv3071   HK1-2024       10000000           0
103     sv3077   HK2-2024        9000000           0
104     sv3078     2024-2        9000000           0
105     sv3078     2025-1     9,000,000      4000000
106     sv3078  HK1- 2025      8.000.000    10000000

[107 rows x 4 columns]


In [74]:
unpaid_count_by_semester = (
    unpaid_or_partial
    .groupby("semester")
    .size()
    .reset_index(name="unpaid_or_partial_students")
)

print(unpaid_count_by_semester)


    semester  unpaid_or_partial_students
0     2024-1                          19
1     2024-2                          13
2     2025-1                          21
3  HK1- 2025                          18
4   HK1-2024                          18
5   HK2-2024                          18
